# Import relevant libraries

In [ ]:
import os
import gdal
import glob
import datetime
from convert_to_pm25 import convert_to_pm25

## Specify my variables

In [ ]:
input_folder= r'D:\MAIAC_Data\Europe\h00v01'
output_folder = r'D:\MAIAC_Data\Europe\h00v01\Projected'
glob_string = '????\MAIAC?AOT*.hdf'
eta_Monthly = r'D:\Annies_Dissertation\MAIAC_Test\vanDs\eta_Monthly.tif'

# Test arguments

input_folder= r'D:\Annies_Dissertation\MAIAC_Test\subset'
output_folder = r'D:\Annies_Dissertation\MAIAC_Test\subset\Projected'
glob_string = '????\MAIAC?AOT*.hdf'
eta_Monthly = r'D:\Annies_Dissertation\MAIAC_Test\vanDs\eta_Monthly.tif'

# Translate Lat and Lon files
- Find out information about the lat lon file
- Using the gdal_translate tool convert the lat and lon hdr files into vrt format

latlon_info = gdal.Info(r'D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf')
print(latlon_info)

gdal.Translate(r"D:\MAIAC_Data\Europe\h00v01\h00v01_lat.vrt", 
               r'HDF4_EOS:EOS_GRID:"D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf":latlon:lat', format = 'VRT' )

gdal.Translate(r"D:\MAIAC_Data\Europe\h00v01\h00v01_lon.vrt", 
               r'HDF4_EOS:EOS_GRID:"D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf":latlon:lon', format = 'VRT' )

# Warp the AOT images
- get a list of names of HDF files
- for each file name in list write data.vrt (change file name in data.vrt for each of the files)
    - use os.system or os.path to write a 'data.vrt' text file in xml language 
    - open("data.vrt")
- run gdal_warp and rename projected*.tif

## Create geoloc.vrt

In [ ]:
geoloc_xml = """<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"{fname}":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>
"""

## Function which takes a MAIAC AOT file and performs geolocation using the associated MAIAC Lat and Lon files, and then warps to specified co-ordinate system

In [ ]:
def geolocate_and_warp(input_filename, output_filename, output_crs='EPSG:27700'):
    with open(r'geoloc_test.vrt', 'w') as myfile:
        myfile.write(geoloc_xml.format(fname=input_filename))
    gdal.Warp(output_filename,
             r'geoloc_test.vrt',
             geoloc = True, srcSRS = 'EPSG:4326', dstSRS = output_crs, 
              srcNodata = -28672, dstNodata = -28672)

## Get the list of all files in the input folder that match the glob string

In [ ]:
filenames = glob.glob(os.path.join(input_folder, glob_string))

## Month variable for AOT_toPM2.5

In [ ]:
def month(AOT_filename):
       
    time_str = os.path.basename(AOT_filename)[17:28]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    mth = time_obj.month
    int_mth = int(mth)

    return int_mth

## For statement which loops over each file and produces the geoloc.vrt and projects the image

In [ ]:
for filename in filenames:
    print ('Processing {fname}'.format(fname=filename))
    output_basename = os.path.join(output_folder,
                                  os.path.relpath(filename,
                                                 start = input_folder))
    output_dir = os.path.dirname(output_basename)
    AOT_filename = output_basename + "_proj.tif"
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    geolocate_and_warp(filename, AOT_filename)
    
    convert_to_pm25(AOT_filename, eta_Monthly, month(AOT_filename), 
                    output_basename + "_PM25.tif", 0.001)